### Preparation - Review raw usage data

In [ ]:
# del df_raw

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from matplotlib import pyplot as plt
import datetime
import numpy as np
from plot_helper import *

CSV_FILE = '../data/GlobalUsageReport.csv'

try:
    df_raw
except NameError:
    df_raw = pd.read_csv(CSV_FILE,encoding="ISO-8859-1",parse_dates=['Usage Period'])

df_raw.groupby('Usage Period')['Value (USD)'].sum().plot()
format_trend_chart()
plt.show()

df_raw.groupby('Usage Period')['Usage Points'].sum().plot()
format_trend_chart()
plt.show()

df_rental = df_raw[(df_raw['Product Family'] == 'Rental') &
                   ((df_raw['Collection Status'] == 'Closed') |
                    (df_raw['Collection Status'] == 'Pending Agg') |
                    (df_raw['Collection Status'] == 'Pending Vendor')                    
                   )]

df_rental.groupby('Usage Period')['Usage Points'].sum().plot()
format_trend_chart()
plt.show()

print(df_raw.columns)


### Preparation - Configure QUARTER_RANGE in defines.py

In [ ]:
from defines import *


df = df_rental[(df_rental['Usage Period']<=END_MONTH) & 
               (df_rental['Usage Period']>=START_MONTH)]

df.groupby('Usage Period')['Value (USD)'].sum().plot()
format_trend_chart()
plt.title("Data Range, {} ~ {}".format(START_MONTH,END_MONTH))
plt.show()



### Total VCPP Revenue & Parnter Cound Trend

In [ ]:
from functools import reduce

plt.close()
fig,ax1 = plt.subplots()
ax2 = ax1.twinx()

rev = df.groupby('Usage Period')['Value (USD)'].sum()
ax1.bar(rev.index,rev.values,10,label='Total Revenue')

cnt = df.groupby('Usage Period')['Service Provider'].unique()
pcnts = list(map(lambda x: len(x), cnt.values))
ax2.plot(cnt.index,pcnts,'o-',c='tomato',label='Partner Count')
ax2.set_ylim(bottom=3500,top=3500*rev.values.max()/rev.values.min())
ax2.set_ylabel('vCAN Partner Count')
ax2.legend(loc="lower right", bbox_to_anchor=(1,0.1))
    
x0=len(cnt.loc[END_MONTH.replace(year=END_MONTH.year-1)])
x1=len(cnt.loc[END_MONTH])
print("YoY Partner Count Growth: +{:.0f}%".format((x1/x0-1)*100))

x0=reduce(lambda x,y:x+y,map(lambda x: rev.loc[x],map(lambda x: x.replace(year=x.year-1),END_QUARTER)))
x1=reduce(lambda x,y:x+y,map(lambda x: rev.loc[x],END_QUARTER))
VCPP_REV_GROWTH = x1/x0-1 
print("YoY Quarter Revenue Growth : +{:.0f}%".format(VCPP_REV_GROWTH*100))

ax1.yaxis.grid(which="major", color='black', linestyle='dashed', linewidth=.3)
ax1.set_yticklabels(['${:,.0f}K'.format(x/1000) for x in ax1.get_yticks()])
ax1.set_ylabel('vCAN Total MRR')
ax1.legend(loc="lower right", bbox_to_anchor=(1,0))

fig = plt.gcf()
fig.autofmt_xdate() 
plt.title("VCPP Revenue & Partner Count")
fig.set_size_inches(10,6)
plt.show()

In [ ]:
def plot_dist(d,caption=''):
    pp = d['Value (USD)']/d['Usage Points']

    pp.plot.hist(bins=10)
    plt.minorticks_on()
    plt.gca().xaxis.grid(True, which='both',linestyle='dashed')
    plt.xlim([0,1.0])
    plt.title("Point Price Distribution {}(Average={:.2f})".format(caption,pp.mean()))
    plt.show()

_df = df[df['Usage Points'] > 0]
_df_fujitsu = _df[_df['Partner Group Name']=='Fujitsu'].copy()

plot_dist(_df)
plot_dist(_df_fujitsu)

# NTT point price trend
_df_ntt = _df[_df['Service Provider']=='NTT Communications Corporation'].copy()
_df_ntt.loc[:,'Point Price'] = pd.Series(_df_ntt['Value (USD)']/_df_ntt['Usage Points'],index=_df_ntt.index)

_df_ntt.groupby('Usage Period')['Point Price'].max().plot()
plt.ylabel('Point Price')
plt.gca().xaxis.grid(True, which='both',linestyle='dashed')
plt.show()




In [ ]:
d = df[df['Usage Points'] > 0]
d = d['Value (USD)']/d['Usage Points']

plt.title("Point Price Distribution Mean={:.2f}".format(d.mean()))
plt.hist(d,bins=10)
plt.show()

### Showcase Partner Monthly Revenue Trend

In [ ]:
showcase_partners = df[(df['Usage Period'] == END_MONTH) &
                       (df['Partner Status'] == 'Showcase')].groupby('Partner Group Name')['Value (USD)'].sum().sort_values(ascending=False)

showcase_partners.plot.pie(startangle=90,counterclock=False)
plt.show()

for partner in showcase_partners.index:
    partner_rev = df[df['Partner Group Name'] == partner].groupby('Usage Period')['Value (USD)'].sum()
    plt.plot(partner_rev.index,partner_rev.values,'o-',label=partner)

format_trend_chart()

plt.title("Showcase Partners Monthly Revenue")
plt.gcf().set_size_inches(3,3)
plt.show()



### Showcase Partner Growth Trend

In [ ]:
def plot_rev_trend(l,data,thick=False):
    if thick:
        plt.plot(data.index,(data.values/data.values[0]-1)*100,'o-',label=l,linewidth=5.0,markersize=9.0)
    else:
        plt.plot(data.index,(data.values/data.values[0]-1)*100,'o-',label=l)

def plot_partner(partner):
    plot_rev_trend(partner,df[df['Partner Group Name']==partner].groupby('Usage Period')['Value (USD)'].sum())

def format_chart(title):
    ax = format_trend_chart()
    ax.set_yticklabels(['+{:,.0f}%'.format(x) for x in ax.get_yticks()])
 #   ax.legend(loc="lower right", bbox_to_anchor=(1,0))
    plt.title(title)

plot_rev_trend('VCPP Total',df.groupby('Usage Period')['Value (USD)'].sum(),True)
plot_rev_trend('Showcase Total',df[df['Partner Status']=='Showcase'].groupby('Usage Period')['Value (USD)'].sum())
format_chart('VCPP Total vs Showcase')
plt.show()

for sp in showcase_partners.index:
    plot_partner(sp)
 
format_chart("Showcase Partners Monthly Revenue Growth")
plt.show()


In [ ]:
import size_vs_growth as sg
    
sg.plot(df[(df['Partner Status']=='Showcase')],
    {'type':['NO_SLICE'],
     'min_rev_growth':10000,
    'show_sp_name':0},group_key='Partner Group Name')

### VCPP Partners Revenue Distribution

In [ ]:
def plot_hist(data_prev,data_last,cutoff,title):
    plt.hist(data_last[:int(len(data_last)*cutoff)],bins=100,alpha=.8,label='2017 Q1')
    plt.hist(data_prev[:int(len(data_last)*cutoff)],bins=100,alpha=.8,label='2016 Q1')

    plt.xlabel("Quarterly Revenue (USD)")
    plt.ylabel("Number of Service Providers")
    plt.title(title)
    ax = plt.gca()
    fig = plt.gcf()
    ax.set_xticklabels(['${:,.0f}K'.format(x/1000) for x in ax.get_xticks()])
    ax.legend(loc="upper right")
    fig.set_size_inches(10,6)
    plt.show()
    
df_last_q = df[(df['Usage Period'] >= LAST_QUARTER[0]) & (df['Usage Period'] <= LAST_QUARTER[2])]
df_prev_q = df[(df['Usage Period'] >= PREV_QUARTER[0]) & (df['Usage Period'] <= PREV_QUARTER[2])]
data_last = df_last_q.groupby('Service Provider')['Value (USD)'].sum().sort_values(ascending=False)
data_prev = df_prev_q.groupby('Service Provider')['Value (USD)'].sum().sort_values(ascending=False)
    
plot_hist(data_prev,data_last,1.0,"Revenue Distribution - All SPs")
plot_hist(data_prev,data_last,0.1,"Revenue Distribution - Top 10% SPs")
    
total_last = df_last_q['Value (USD)'].sum()
print("Total SP Count: {}".format(len(data_last)))
print("Top {} SPs Contribution: {:.0f}%".format(10,data_last[:10].sum()/total_last*100))
    
plt.pie(data_last.values,startangle=90,counterclock=False)
plt.title('Revenue Distribution')
plt.gcf().set_size_inches(3,3)
plt.show()


### Showcase Partner - Group Companies Revenue Trend

In [ ]:
def partner_rev_trend(group,key='Value (USD)'):
    # sort by revenue size at the last month
    sp_group = df[df['Partner Group Name'] == group]
    sp_sorted = sp_group.groupby('Service Provider')[key].sum().sort_values(ascending=False)

    sp_sorted.plot.pie(startangle=90,counterclock=False)
    plt.gcf().set_size_inches(6,6)
    plt.show() 
    
    # get the series of usage period
    index = sp_group.groupby('Usage Period')[key].sum().index
    
    prev_bar = 0
    for sp in sp_sorted.index:
        rev_by_period = df[df['Service Provider'] == sp].groupby('Usage Period')[key].sum() 
        for up in index:
            if not up in rev_by_period.keys():
                rev_by_period.set_value(up,0.0)
        rev_by_period = rev_by_period.sort_index()
 
        plt.bar(rev_by_period.index,rev_by_period.values,width=20,label=sp,bottom=prev_bar)
        prev_bar += rev_by_period.values
        
    format_trend_chart()

    plt.title('{} Group Revenue Trend Chart'.format(group))
    plt.gcf().set_size_inches(8,6)
    plt.show()
    
partner_rev_trend('NTT','Usage Points')
partner_rev_trend('Fujitsu','Usage Points')


### Revenue Size vs Growth

In [ ]:
import size_vs_growth as sg

sg.plot(df,
     {'type':['GEO','RANK'],
      'time_frame': 'LAST_12',
      'min_rev_growth':10000,
      'show_sp_name':500000})
    
sg.plot(df[(df['Country']=='Japan')],
    {'type':['NO_SLICE'],
     'time_frame': 'LAST_12',     
     'min_rev_growth':10000,
    'show_sp_name':100000})